![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor


# Import any additional modules and start coding below

In [2]:
#lectura de datos
df = pd.read_csv("rental_info.csv")

#cambio de columnas a datetime
df.rental_date = pd.to_datetime(df.rental_date)
df.return_date = pd.to_datetime(df.return_date)

#columna rental length days
df['rental_length_days'] = (df.return_date - df.rental_date).dt.days
df['rental_length_days']

0        3
1        2
2        7
3        2
4        4
        ..
15856    6
15857    4
15858    9
15859    8
15860    6
Name: rental_length_days, Length: 15861, dtype: int64

In [3]:
#dummy columns
df['deleted_scenes'] = df.special_features.str.contains("Deleted Scenes")
df['behind_the_scenes'] = df.special_features.str.contains("Behind the Scenes")
df[['deleted_scenes', 'behind_the_scenes']] = df[['deleted_scenes', 'behind_the_scenes']].astype(int)

In [4]:
#separacion en train y test
X = df[['amount', 'release_year', 'rental_rate', 'length', 'replacement_cost',
                      'NC-17', 'PG', 'PG-13', 'R', 'amount_2', 'length_2', 'rental_rate_2', 'deleted_scenes',
                       'behind_the_scenes']]
y = df['rental_length_days']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 9)

In [109]:
linear_params = {
    # Los modelos lineales no tienen muchos hiperparámetros para ajustar
}

rf_params = {
    'n_estimators': [10, 50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

xgb_params = {
    'n_estimators': [100, 500, 1000],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'subsample': [0.5, 0.75, 1],
    'colsample_bytree': [0.5, 0.75, 1],
    'gamma': [0, 0.1, 0.3],
    'reg_alpha': [0, 0.1, 0.5],
    'reg_lambda': [0, 0.1, 0.5],
    'min_child_weight': [1, 5, 10]
}


rs = RandomizedSearchCV(
    estimator = XGBRegressor(),
    param_distributions = xgb_params,
    scoring = mean_squared_error,
    n_jobs = -1,
    cv =5,
    n_iter = 20,
)

rs.fit(X_test, y_test)

In [110]:
y_hat = rs.best_estimator_.predict(X_test)
best_mse = mean_squared_error(y_test, y_hat)
print(f"mean squared error {best_mse:2f}")
best_model = rs.best_estimator_

mean squared error 1.701903
